In [ ]:
root_dir = "/content/drive/MyDrive/mah_ws/ai_prj/prj_ImageSuper-Resolution"
%cd /content/drive/MyDrive/mah_ws/ai_prj/prj_ImageSuper-Resolution

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/mah_ws/ai_prj/prj_ImageSuper-Resolution


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from model import edsr
from libs import agumentation
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [ ]:
def PSNR(super_resolution, high_resolution):
    """Compute the peak signal-to-noise ratio, measures quality of image."""
    # Max value of pixel is 255
    psnr_value = tf.image.psnr(high_resolution, super_resolution, max_val=255)[0]
    return psnr_value

In [ ]:
## Load data
ds_train = tfds.load(f'div2k/bicubic_x{scale}',split='train[:75%]',as_supervised=True)
ds_valid = tfds.load(f'div2k/bicubic_x{scale}',split='train[75%:]',as_supervised=True)

ds_test = tfds.load(f'div2k/bicubic_x{scale}',split='validation',as_supervised=True)

In [ ]:
train = agumentation.preprocessing(ds_train,scale=scale)
valid = agumentation.preprocessing(ds_valid,scale=scale,batch_size=1,repeat_count=1,random_transform=False)

test  = agumentation.preprocessing(ds_test,scale=scale,batch_size=1,repeat_count=1,random_transform=False)

In [ ]:
print(f"{name_pre}")
model = tf.keras.models.load_model(f"{root_dir}/save_model/edsr/shuffle/{name_pre}.h5",custom_objects={'PSNR':PSNR})
model.load_weights(f"{root_dir}/checkpoint/edsr/shuffle/{name_pre}.h5")

model.fit(train,validation_data=valid,epochs=epochs,verbose=True, callbacks=[checkpoint,reducelr],steps_per_epoch=steps_epoch,validation_steps=1)

print(f'{ws_dir}/save_model/edsr/{name_pre}_extra100.h5')
model.save(f'{ws_dir}/save_model/edsr/{name_pre}_extra100.h5')


log = pd.DataFrame(model.history.history)
    
log[['val_loss']].plot()
log[['val_PSNR']].plot()

## save hist
with open(f'{root_dir}/save_model/edsr/{name_pre}_extra100.h5.json', mode='wt') as f:
    log.to_json(f)
log

model.evaluate(test)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/50
100/100 [==============================] - 208s 2s/step - loss: 4.0814 - PSNR: 37.6452 - val_loss: 1.1289 - val_PSNR: 43.0636 - lr: 1.0000e-05
Epoch 2/50
100/100 [==============================] - 142s 1s/step - loss: 4.2838 - PSNR: 36.9200 - val_loss: 1.1420 - val_PSNR: 42.9126 - lr: 1.0000e-05
Epoch 3/50
100/100 [==============================] - 135s 1s/step - loss: 4.0029 - PSNR: 36.9740 - val_loss: 1.1293 - val_PSNR: 43.1247 - lr: 1.0000e-05
Epoch 4/50
100/100 [==============================] - ETA: 0s - loss: 4.0042 - PSNR: 36.5589
Epoch 4: ReduceLROnPlateau reducing learning rate to 9.048373613040894e-06.
100/100 [==============================] - 135s 1s/step - loss: 4.0042 - PSNR: 36.5589 - val_loss: 1.1392 - val_PSNR: 42.8591 - lr: 1.0000e-05
Epoch 5/50
100/100 [==============================] - 157s 2s/step - loss: 4.1209 - PSNR: 36.7494 - val_loss: 1.1220 - val_PSNR: 43.0623 - lr: 9.0484e-06
Epoch 6/50
100/100 [==============================] - 129s 1s/step - los